# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [4]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./audios/fourier_32/{}'.format(chunk_name), format="mp3")
            
decompose_files('./audios/audios_base/', glob('./audios/audios_base' + '/*'))

In [ ]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=32)[:8000 // 2])
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, 
                         D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], D[10], D[11], 
                         D[12], D[13], D[14], D[15], D[16], D[17], 
                         D[18], D[19], D[20], D[21], D[22], D[23],
                         D[24], D[25], D[26], D[27], D[28], D[29],
                         D[30], D[31], momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
                                           'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
                                           'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
                                           'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
                                           'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
                                           'Fourier31', 'Fourier32', 'momento'])

data = get_features('./audios/fourier_32/', glob('./audios/fourier_32' + '/*.wav'))

In [5]:
data.shape

(14611, 40)

En este paso intermedio, dado que hay una pequeña cantidad de audios cuyas features son todas cero, los elimino para no ensuciar el modelo.

In [6]:
data = data[data['Fourier5'] != 0].reset_index(drop=True)

In [7]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe.head())
    display(dataframe.describe())
    
show_df_info(data)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 572360.'

------------------------------------------------------------------------


'Its shape is (14309, 40).'

------------------------------------------------------------------------


"Its features are: ['mfcc', 'scem', 'scom', 'srom', 'sbwm', 'tempo', 'rmse', 'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10', 'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15', 'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20', 'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25', 'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30', 'Fourier31', 'Fourier32', 'momento']."

------------------------------------------------------------------------
The data types of columns are: mfcc         float64
scem         float64
scom         float64
srom         float64
sbwm         float64
tempo        float64
rmse         float64
Fourier1     float64
Fourier2     float64
Fourier3     float64
Fourier4     float64
Fourier5     float64
Fourier6     float64
Fourier7     float64
Fourier8     float64
Fourier9     float64
Fourier10    float64
Fourier11    float64
Fourier12    float64
Fourier13    float64
Fourier14    float64
Fourier15    float64
Fourier16    float64
Fourier17    float64
Fourier18    float64
Fourier19    float64
Fourier20    float64
Fourier21    float64
Fourier22    float64
Fourier23    float64
Fourier24    float64
Fourier25    float64
Fourier26    float64
Fourier27    float64
Fourier28    float64
Fourier29    float64
Fourier30    float64
Fourier31    float64
Fourier32    float64
momento       object
dtype: object
------------------------------------------

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-19.602940,1016.689071,28.116497,1344.604492,526.793735,117.187500,0.082146,0.009263,0.012996,0.012584,...,0.006196,0.007697,0.008855,0.012467,0.022492,0.063204,0.154885,0.012584,0.012996,Timbre
1,-23.112227,1361.218925,25.115322,2646.606445,1025.292241,117.187500,0.055449,0.091122,0.116129,0.504887,...,0.284429,0.137750,0.123100,0.141838,0.087461,0.039865,0.118851,0.504887,0.116129,Cena
2,4.153023,662.289054,21.617422,1497.436523,832.083155,156.250000,0.240111,5.745226,4.179618,0.842350,...,0.145524,0.233634,0.307225,0.486006,0.661933,0.242134,0.867832,0.842350,4.179618,Lavadora
3,-15.240392,1539.737568,22.446293,2865.844727,1002.431942,133.928571,0.113920,0.667184,0.231604,0.799329,...,0.379264,0.510500,0.445284,0.477069,0.804105,0.829143,1.475118,0.799329,0.231604,Secador_pelo
4,6.963927,582.127562,19.931546,1348.144531,798.607193,133.928571,0.212797,2.974887,0.945422,0.571051,...,0.152262,0.090392,0.157818,0.212591,0.090181,0.243442,0.554560,0.571051,0.945422,Aire_acondicionado


,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier23,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32
count,14309.000000,14309.000000,14309.000000,14309.000000,14309.000000,14309.000000,1.430900e+04,1.430900e+04,1.430900e+04,1.430900e+04,...,1.430900e+04,14309.000000,14309.000000,1.430900e+04,14309.000000,1.430900e+04,14309.000000,1.430900e+04,1.430900e+04,1.430900e+04
mean,-8.971953,1254.151396,22.267809,2249.303162,892.667099,130.348089,8.841907e-02,9.235651e-01,8.207713e-01,6.463853e-01,...,1.471023e-01,0.151666,0.167978,2.257706e-01,0.236699,2.674153e-01,0.323432,4.386883e-01,6.463853e-01,8.207713e-01
std,10.089162,554.546872,3.258389,810.205158,197.193087,19.883178,8.043133e-02,1.799353e+00,1.226849e+00,1.373220e+00,...,1.939466e-01,0.202599,0.245692,3.550661e-01,0.330550,3.885057e-01,0.462326,5.672125e-01,1.373220e+00,1.226849e+00
min,-56.459336,168.275230,1.698515,291.992188,101.931160,37.500000,5.919943e-08,5.073782e-07,5.302750e-07,9.490217e-07,...,8.521026e-07,0.000001,0.000001,5.838372e-07,0.000002,8.068922e-07,0.000001,6.708018e-07,9.490217e-07,5.302750e-07
25%,-16.452749,798.831594,19.927511,1515.258789,805.112993,117.187500,1.969259e-02,3.763595e-02,5.440067e-02,4.146728e-02,...,2.184711e-02,0.023131,0.025481,2.781803e-02,0.034783,3.334841e-02,0.032738,3.284250e-02,4.146728e-02,5.440067e-02
50%,-7.939312,1280.946945,21.832554,2380.004883,927.216385,133.928571,6.710561e-02,1.933344e-01,2.683205e-01,2.511623e-01,...,7.997130e-02,0.085392,0.095736,1.109115e-01,0.126726,1.272525e-01,0.151234,2.159520e-01,2.511623e-01,2.683205e-01
75%,-3.015536,1738.380026,23.659746,2838.134766,1038.395741,133.928571,1.351235e-01,8.397143e-01,1.068091e+00,7.669710e-01,...,1.926018e-01,0.204286,0.222277,2.617446e-01,0.303950,3.426385e-01,0.432991,6.334587e-01,7.669710e-01,1.068091e+00
max,8.775053,2707.461116,51.275412,3609.375000,1299.501746,312.500000,5.108476e-01,1.580141e+01,1.147088e+01,1.399953e+01,...,1.733549e+00,2.803029,4.165502,3.959535e+00,3.784848,3.763040e+00,5.427896,5.434906e+00,1.399953e+01,1.147088e+01


In [9]:
data.to_csv('data_fourier.csv', index=False)